# Clase Práctica 04

# Neural Network - Redes Neuronales Artificiales

# Clasificación binaria de un Sonar

El conjunto de datos que usaremos en esta clase práctica es la base de datos de un sonar. La base de datos describe los retornos del sonar rebotando en diferentes superficies. Las 60 variables de entrada son la fuerza de los rebotes con diferentes ángulos. Es un problema de clasificación binaria que requiere un modelo para diferenciar si el material que sensa el sonar son rocas o metal.

De la base de datos se observa que todas las variables son continuas y generalmente están en el rango de 0 a 1. La variable de salida es un string "M" para metal y "R" para roca, que deberá convertirse (codificarse) en números enteros 1 y 0. El conjunto de datos contiene 208 observaciones. 

La idea de usar esta base de datos, es que es un problema conocido y que ha sido estudiado. Esto significa que tenemos una idea de la habilidad esperada del modelo. Mediante la validación cruzada, una red neuronal debería poder alcanzar un rendimiento de aproximadamente el 84% con un límite superior en la precisión para modelos personalizados en aproximadamente el 88%.


# Clasificación binaria del Sonar

Comenzaremos importando todas las clases y funciones que necesitaremos. Además, podemos inicializar el generador de números aleatorios seed para asegurarnos de que siempre obtengamos los mismos resultados al ejecutar este código. Esto ayudará si estamos depurando.


In [1]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
# seed
seed = 7
numpy.random.seed(seed)

Using TensorFlow backend.


Ahora podemos cargar la base de datos utilizando Pandas y separar en columnas en 60 variables de entrada (X) y 1 variable de salida (Y). Utilizamos la librería Pandas para cargar los datos porque maneja caracteres (la variable de salida), mientras que intentar cargar los datos directamente usando, por ejemplo, NumPy sería más difícil.

La variable de salida es una letra (caracter, "M" o "R"), debemos convertirla a valores enteros 0 y 1. Podemos hacer esto usando la clase LabelEncoder() de Scikit-Learn. Como habíamos visto en las prácticas anteriores, esta función realiza la codificación mediante la función encoder.fit() y luego aplica la nueva codificación en la variable de salida usando encoder.transform().  

In [2]:
## base de datos
dataframe = pandas.read_csv("sonar.csv", header=None)
dataset = dataframe.values
# entrada y salidas 
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

# codificación
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

dummy_y = encoded_Y   #codificada con indices enteros 0, 1
print("Codificando la salida:")
print(dummy_y)
print("Forma de los datos de entrada:")
print("ejemplos, atributos")
print(X.shape)
print("Forma de las etiquetas:")
print(Y.shape)

Codificando la salida:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Forma de los datos de entrada:
ejemplos, atributos
(208, 60)
Forma de las etiquetas:
(208,)



Vamos a crear nuestro modelo y usaremos Scikit-Learn para evaluar el modelo utilizando la validación cruzada de k-fold. Para usar los modelos Keras con Scikit-Learn, debemos usar el contenedor KerasClassifier, de la misma forma que en la práctica anterior. 

La clase KerasClassifier toma una función que crea y devuelve nuestro modelo de red neuronal. Se deben ingresar los argumentos para pasar a la función de ajuste fit(), como el número de épocas y el tamaño del lote. Vamos a empezar definiendo la función que crea nuestro modelo, llamada sonar_model(). Nuestro modelo tendrá una única capa oculta totalmente conectada con el mismo número de neuronas que las variables de entrada. Luego podemos crear otras redes neuronales para analizar el problema. 

Los pesos se inicializan utilizando un pequeño número aleatorio gaussiano ("normal"). Se utiliza la función de activación de rectificación (ReLU). La capa de salida contiene una sola neurona para hacer predicciones (**clasificación binaria**). Se utiliza la función de activación sigmoide para producir una salida de probabilidad en el rango de 0 a 1 que se puede convertir fácil y automáticamente a valores de clase nítidos. Además, se usa la función de pérdida logarítmica (**binary crossentropy**) durante el entrenamiento, que es la función de pérdida preferida para los problemas de clasificación binaria. El modelo también utiliza como optimizador el algoritmo Adam para el descenso del gradiente y las métricas de accuracy se recopilarán cuando se entrene el modelo.

In [3]:
# definir funcion para el modelo de red neuronal
def sonar_model():
    # modelo
    model = Sequential()
    model.add(Dense(60, input_dim=60, init='normal', activation='relu'))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    # compilar
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

Ahora es el momento de evaluar este modelo utilizando una validación cruzada con Scikit-Learn. Le pasamos el número de épocas de entrenamiento y el batch_size al KerasClassifier. Se utiliza un k-fold de 10 veces para la validación cruzada.

In [4]:
# evaluar el modelo con k-fold

estimator = KerasClassifier(build_fn=sonar_model, epochs=100, batch_size=5)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(60, input_dim=60, activation="relu", kernel_initializer="normal")`
  """
C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`
  


Epoch 1/100
187/187 [==============================] - 3s 17ms/step - loss: 0.6909 - acc: 0.4759
Epoch 2/100
187/187 [==============================] - 0s 1ms/step - loss: 0.6777 - acc: 0.5882
Epoch 3/100
187/187 [==============================] - 0s 1ms/step - loss: 0.6660 - acc: 0.6257
Epoch 4/100
187/187 [==============================] - 0s 1ms/step - loss: 0.6502 - acc: 0.6578
Epoch 5/100
187/187 [==============================] - 0s 1ms/step - loss: 0.6253 - acc: 0.6898
Epoch 6/100
187/187 [==============================] - 0s 1ms/step - loss: 0.6008 - acc: 0.7540
Epoch 7/100
187/187 [==============================] - 0s 1ms/step - loss: 0.5707 - acc: 0.7487
Epoch 8/100
187/187 [==============================] - 0s 1ms/step - loss: 0.5473 - acc: 0.7594
Epoch 9/100
187/187 [==============================] - 0s 1ms/step - loss: 0.5249 - acc: 0.7701
Epoch 10/100
187/187 [==============================] - 0s 923us/step - loss: 0.5228 - acc: 0.7701
Epoch 11/100
187/187 [==============

187/187 [==============================] - 0s 752us/step - loss: 0.3224 - acc: 0.8449
Epoch 65/100
187/187 [==============================] - 0s 709us/step - loss: 0.3149 - acc: 0.8717
Epoch 66/100
187/187 [==============================] - 0s 736us/step - loss: 0.3166 - acc: 0.8610
Epoch 67/100
187/187 [==============================] - 0s 715us/step - loss: 0.3109 - acc: 0.8770
Epoch 68/100
187/187 [==============================] - 0s 768us/step - loss: 0.3108 - acc: 0.8503
Epoch 69/100
187/187 [==============================] - 0s 795us/step - loss: 0.3048 - acc: 0.8663
Epoch 70/100
187/187 [==============================] - 0s 720us/step - loss: 0.3052 - acc: 0.8717
Epoch 71/100
187/187 [==============================] - 0s 720us/step - loss: 0.3072 - acc: 0.8663
Epoch 72/100
187/187 [==============================] - 0s 704us/step - loss: 0.2987 - acc: 0.8824
Epoch 73/100
187/187 [==============================] - 0s 720us/step - loss: 0.3033 - acc: 0.8556
Epoch 74/100
187/187 [=

187/187 [==============================] - 0s 741us/step - loss: 0.4063 - acc: 0.8182
Epoch 29/100
187/187 [==============================] - 0s 709us/step - loss: 0.3962 - acc: 0.8075
Epoch 30/100
187/187 [==============================] - 0s 768us/step - loss: 0.4091 - acc: 0.7807
Epoch 31/100
187/187 [==============================] - 0s 715us/step - loss: 0.3910 - acc: 0.8075
Epoch 32/100
187/187 [==============================] - 0s 725us/step - loss: 0.3828 - acc: 0.8396
Epoch 33/100
187/187 [==============================] - 0s 768us/step - loss: 0.3801 - acc: 0.8342
Epoch 34/100
187/187 [==============================] - 0s 709us/step - loss: 0.3755 - acc: 0.8396
Epoch 35/100
187/187 [==============================] - 0s 741us/step - loss: 0.3799 - acc: 0.8021
Epoch 36/100
187/187 [==============================] - 0s 747us/step - loss: 0.3706 - acc: 0.8128
Epoch 37/100
187/187 [==============================] - 0s 720us/step - loss: 0.3802 - acc: 0.8396
Epoch 38/100
187/187 [=

187/187 [==============================] - 0s 736us/step - loss: 0.2350 - acc: 0.9091
Epoch 93/100
187/187 [==============================] - 0s 720us/step - loss: 0.2344 - acc: 0.9198
Epoch 94/100
187/187 [==============================] - 0s 741us/step - loss: 0.2477 - acc: 0.8877
Epoch 95/100
187/187 [==============================] - 0s 731us/step - loss: 0.2435 - acc: 0.8770
Epoch 96/100
187/187 [==============================] - 0s 715us/step - loss: 0.2337 - acc: 0.9037
Epoch 97/100
187/187 [==============================] - 0s 725us/step - loss: 0.2271 - acc: 0.9251
Epoch 98/100
187/187 [==============================] - 0s 731us/step - loss: 0.2182 - acc: 0.9198
Epoch 99/100
187/187 [==============================] - 0s 725us/step - loss: 0.2400 - acc: 0.9037
Epoch 100/100
21/21 [==============================] - 0s 5ms/step
Epoch 1/100
187/187 [==============================] - 1s 3ms/step - loss: 0.6888 - acc: 0.5455
Epoch 2/100
187/187 [==============================] - 0s 

187/187 [==============================] - 0s 736us/step - loss: 0.3011 - acc: 0.8824
Epoch 57/100
187/187 [==============================] - 0s 709us/step - loss: 0.3038 - acc: 0.8770
Epoch 58/100
187/187 [==============================] - 0s 725us/step - loss: 0.2930 - acc: 0.8824
Epoch 59/100
187/187 [==============================] - 0s 725us/step - loss: 0.2885 - acc: 0.8877
Epoch 60/100
187/187 [==============================] - 0s 720us/step - loss: 0.2888 - acc: 0.8770
Epoch 61/100
187/187 [==============================] - 0s 736us/step - loss: 0.2824 - acc: 0.8984
Epoch 62/100
187/187 [==============================] - 0s 709us/step - loss: 0.2806 - acc: 0.8770
Epoch 63/100
187/187 [==============================] - 0s 736us/step - loss: 0.2998 - acc: 0.8824
Epoch 64/100
187/187 [==============================] - 0s 768us/step - loss: 0.2846 - acc: 0.8663
Epoch 65/100
187/187 [==============================] - 0s 725us/step - loss: 0.2774 - acc: 0.8930
Epoch 66/100
187/187 [=

188/188 [==============================] - 0s 727us/step - loss: 0.4004 - acc: 0.8351
Epoch 21/100
188/188 [==============================] - 0s 711us/step - loss: 0.3917 - acc: 0.8404
Epoch 22/100
188/188 [==============================] - 0s 737us/step - loss: 0.3963 - acc: 0.8298
Epoch 23/100
188/188 [==============================] - 0s 727us/step - loss: 0.3905 - acc: 0.8404
Epoch 24/100
188/188 [==============================] - 0s 721us/step - loss: 0.3876 - acc: 0.8191
Epoch 25/100
188/188 [==============================] - 0s 753us/step - loss: 0.3834 - acc: 0.8138
Epoch 26/100
188/188 [==============================] - 0s 727us/step - loss: 0.3829 - acc: 0.8245
Epoch 27/100
188/188 [==============================] - 0s 732us/step - loss: 0.3674 - acc: 0.8511
Epoch 28/100
188/188 [==============================] - 0s 796us/step - loss: 0.3656 - acc: 0.8564
Epoch 29/100
188/188 [==============================] - 0s 838us/step - loss: 0.3669 - acc: 0.8351
Epoch 30/100
188/188 [=

Epoch 85/100
188/188 [==============================] - 0s 806us/step - loss: 0.2462 - acc: 0.8989
Epoch 86/100
188/188 [==============================] - 0s 796us/step - loss: 0.2453 - acc: 0.9043
Epoch 87/100
188/188 [==============================] - 0s 775us/step - loss: 0.2512 - acc: 0.9043
Epoch 88/100
188/188 [==============================] - 0s 759us/step - loss: 0.2572 - acc: 0.8936
Epoch 89/100
188/188 [==============================] - 0s 775us/step - loss: 0.2410 - acc: 0.9043
Epoch 90/100
188/188 [==============================] - 0s 891us/step - loss: 0.2459 - acc: 0.9043
Epoch 91/100
188/188 [==============================] - 0s 737us/step - loss: 0.2325 - acc: 0.9043
Epoch 92/100
188/188 [==============================] - 0s 737us/step - loss: 0.2401 - acc: 0.8830
Epoch 93/100
188/188 [==============================] - 0s 732us/step - loss: 0.2355 - acc: 0.9149
Epoch 94/100
188/188 [==============================] - 0s 721us/step - loss: 0.2310 - acc: 0.9149
Epoch 95/1

# Código completo para revisión

In [5]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
# seed
seed = 7
numpy.random.seed(seed)

## base de datos
dataframe = pandas.read_csv("sonar.csv", header=None)
dataset = dataframe.values
# entradas y salidas
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

# codificación
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

dummy_y = encoded_Y
# print(dummy_y)

# función para crear el modelo
def sonar():
    # modelo
    model = Sequential()
    model.add(Dense(60, input_dim=60, init='normal', activation='relu'))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    # compilar
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

estimator = KerasClassifier(build_fn=sonar, epochs=100, batch_size=5)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:34: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(60, input_dim=60, activation="relu", kernel_initializer="normal")`
C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`


Epoch 1/100
187/187 [==============================] - 1s 5ms/step - loss: 0.6909 - acc: 0.4759
Epoch 2/100
187/187 [==============================] - 0s 843us/step - loss: 0.6777 - acc: 0.5882
Epoch 3/100
187/187 [==============================] - 0s 859us/step - loss: 0.6660 - acc: 0.6257
Epoch 4/100
187/187 [==============================] - 0s 837us/step - loss: 0.6502 - acc: 0.6578
Epoch 5/100
187/187 [==============================] - 0s 848us/step - loss: 0.6253 - acc: 0.6898
Epoch 6/100
187/187 [==============================] - 0s 837us/step - loss: 0.6008 - acc: 0.7540
Epoch 7/100
187/187 [==============================] - 0s 816us/step - loss: 0.5707 - acc: 0.7487
Epoch 8/100
187/187 [==============================] - 0s 821us/step - loss: 0.5473 - acc: 0.7594
Epoch 9/100
187/187 [==============================] - 0s 816us/step - loss: 0.5249 - acc: 0.7701
Epoch 10/100
187/187 [==============================] - 0s 821us/step - loss: 0.5228 - acc: 0.7701
Epoch 11/100
187/187 

187/187 [==============================] - 0s 741us/step - loss: 0.3149 - acc: 0.8717
Epoch 66/100
187/187 [==============================] - 0s 757us/step - loss: 0.3166 - acc: 0.8610
Epoch 67/100
187/187 [==============================] - 0s 763us/step - loss: 0.3109 - acc: 0.8770
Epoch 68/100
187/187 [==============================] - 0s 736us/step - loss: 0.3108 - acc: 0.8503
Epoch 69/100
187/187 [==============================] - 0s 752us/step - loss: 0.3048 - acc: 0.8663
Epoch 70/100
187/187 [==============================] - 0s 763us/step - loss: 0.3052 - acc: 0.8717
Epoch 71/100
187/187 [==============================] - 0s 773us/step - loss: 0.3072 - acc: 0.8663
Epoch 72/100
187/187 [==============================] - 0s 757us/step - loss: 0.2987 - acc: 0.8824
Epoch 73/100
187/187 [==============================] - 0s 768us/step - loss: 0.3033 - acc: 0.8556
Epoch 74/100
187/187 [==============================] - 0s 779us/step - loss: 0.3066 - acc: 0.8717
Epoch 75/100
187/187 [=

187/187 [==============================] - 0s 816us/step - loss: 0.3962 - acc: 0.8075
Epoch 30/100
187/187 [==============================] - 0s 773us/step - loss: 0.4091 - acc: 0.7807
Epoch 31/100
187/187 [==============================] - 0s 789us/step - loss: 0.3910 - acc: 0.8075
Epoch 32/100
187/187 [==============================] - 0s 768us/step - loss: 0.3828 - acc: 0.8396
Epoch 33/100
187/187 [==============================] - 0s 779us/step - loss: 0.3801 - acc: 0.8342
Epoch 34/100
187/187 [==============================] - 0s 763us/step - loss: 0.3755 - acc: 0.8396
Epoch 35/100
187/187 [==============================] - 0s 800us/step - loss: 0.3799 - acc: 0.8021
Epoch 36/100
187/187 [==============================] - 0s 779us/step - loss: 0.3706 - acc: 0.8128
Epoch 37/100
187/187 [==============================] - 0s 779us/step - loss: 0.3802 - acc: 0.8396
Epoch 38/100
187/187 [==============================] - 0s 747us/step - loss: 0.3745 - acc: 0.8075
Epoch 39/100
187/187 [=

187/187 [==============================] - 0s 880us/step - loss: 0.2350 - acc: 0.9091
Epoch 93/100
187/187 [==============================] - 0s 837us/step - loss: 0.2344 - acc: 0.9198
Epoch 94/100
187/187 [==============================] - 0s 939us/step - loss: 0.2477 - acc: 0.8877
Epoch 95/100
187/187 [==============================] - 0s 859us/step - loss: 0.2435 - acc: 0.8770
Epoch 96/100
187/187 [==============================] - 0s 805us/step - loss: 0.2337 - acc: 0.9037
Epoch 97/100
187/187 [==============================] - 0s 955us/step - loss: 0.2271 - acc: 0.9251
Epoch 98/100
187/187 [==============================] - 0s 757us/step - loss: 0.2182 - acc: 0.9198
Epoch 99/100
187/187 [==============================] - 0s 747us/step - loss: 0.2400 - acc: 0.9037
Epoch 100/100
21/21 [==============================] - 0s 12ms/step
Epoch 1/100
187/187 [==============================] - 1s 5ms/step - loss: 0.6888 - acc: 0.5455
Epoch 2/100
187/187 [==============================] - 0s

187/187 [==============================] - 0s 800us/step - loss: 0.3011 - acc: 0.8824
Epoch 57/100
187/187 [==============================] - 0s 816us/step - loss: 0.3038 - acc: 0.8770
Epoch 58/100
187/187 [==============================] - 0s 800us/step - loss: 0.2930 - acc: 0.8824
Epoch 59/100
187/187 [==============================] - 0s 832us/step - loss: 0.2885 - acc: 0.8877
Epoch 60/100
187/187 [==============================] - 0s 789us/step - loss: 0.2888 - acc: 0.8770
Epoch 61/100
187/187 [==============================] - 0s 800us/step - loss: 0.2824 - acc: 0.8984
Epoch 62/100
187/187 [==============================] - 0s 805us/step - loss: 0.2806 - acc: 0.8770
Epoch 63/100
187/187 [==============================] - 0s 800us/step - loss: 0.2998 - acc: 0.8824
Epoch 64/100
187/187 [==============================] - 0s 811us/step - loss: 0.2846 - acc: 0.8663
Epoch 65/100
187/187 [==============================] - 0s 805us/step - loss: 0.2774 - acc: 0.8930
Epoch 66/100
187/187 [=

188/188 [==============================] - 0s 854us/step - loss: 0.4004 - acc: 0.8351
Epoch 21/100
188/188 [==============================] - 0s 780us/step - loss: 0.3917 - acc: 0.8404
Epoch 22/100
188/188 [==============================] - 0s 881us/step - loss: 0.3963 - acc: 0.8298
Epoch 23/100
188/188 [==============================] - 0s 881us/step - loss: 0.3905 - acc: 0.8404
Epoch 24/100
188/188 [==============================] - 0s 769us/step - loss: 0.3876 - acc: 0.8191
Epoch 25/100
188/188 [==============================] - 0s 817us/step - loss: 0.3834 - acc: 0.8138
Epoch 26/100
188/188 [==============================] - 0s 796us/step - loss: 0.3829 - acc: 0.8245
Epoch 27/100
188/188 [==============================] - 0s 753us/step - loss: 0.3674 - acc: 0.8511
Epoch 28/100
188/188 [==============================] - 0s 774us/step - loss: 0.3656 - acc: 0.8564
Epoch 29/100
188/188 [==============================] - 0s 785us/step - loss: 0.3669 - acc: 0.8351
Epoch 30/100
188/188 [=

188/188 [==============================] - 0s 870us/step - loss: 0.6789 - acc: 0.6170
Epoch 3/100
188/188 [==============================] - 0s 891us/step - loss: 0.6687 - acc: 0.6277
Epoch 4/100
188/188 [==============================] - 0s 838us/step - loss: 0.6499 - acc: 0.6064
Epoch 5/100
188/188 [==============================] - 0s 822us/step - loss: 0.6359 - acc: 0.7074
Epoch 6/100
188/188 [==============================] - 0s 817us/step - loss: 0.6088 - acc: 0.6862
Epoch 7/100
188/188 [==============================] - 0s 817us/step - loss: 0.5896 - acc: 0.7500
Epoch 8/100
188/188 [==============================] - 0s 833us/step - loss: 0.5652 - acc: 0.7234
Epoch 9/100
188/188 [==============================] - 0s 817us/step - loss: 0.5445 - acc: 0.7766
Epoch 10/100
188/188 [==============================] - 0s 822us/step - loss: 0.5222 - acc: 0.7872
Epoch 11/100
188/188 [==============================] - 0s 807us/step - loss: 0.5123 - acc: 0.7766
Epoch 12/100
188/188 [========

Epoch 85/100
188/188 [==============================] - 0s 796us/step - loss: 0.2462 - acc: 0.8989
Epoch 86/100
188/188 [==============================] - 0s 817us/step - loss: 0.2453 - acc: 0.9043
Epoch 87/100
188/188 [==============================] - 0s 806us/step - loss: 0.2512 - acc: 0.9043
Epoch 88/100
188/188 [==============================] - 0s 812us/step - loss: 0.2572 - acc: 0.8936
Epoch 89/100
188/188 [==============================] - 0s 1ms/step - loss: 0.2410 - acc: 0.9043
Epoch 90/100
188/188 [==============================] - 0s 897us/step - loss: 0.2459 - acc: 0.9043
Epoch 91/100
188/188 [==============================] - 0s 801us/step - loss: 0.2325 - acc: 0.9043
Epoch 92/100
188/188 [==============================] - 0s 801us/step - loss: 0.2401 - acc: 0.8830
Epoch 93/100
188/188 [==============================] - 0s 838us/step - loss: 0.2355 - acc: 0.9149
Epoch 94/100
188/188 [==============================] - 0s 838us/step - loss: 0.2310 - acc: 0.9149
Epoch 95/100

# Mejorar el rendimiento con el procesamiento de los datos

Es una buena práctica procesarr sus datos antes de generar el modelo de red. Un esquema de procesamiento de datos para construir modelos de redes neuronales es la estandarización (normalización). Aquí es donde los datos se reajustan de manera tal que el valor medio para cada atributo es 0 y la desviación estándar es 1. 

Podemos usar Scikit-Learn para realizar la estandarización de nuestr base de datos de Sonar utilizando la clase StandardScaler. En lugar de realizar la estandarización en todo el conjunto de datos, se realiza solo a los datos de entrenamiento durante la ejecución del la validación cruzada k fold. Así evalua el modelo con datos no vistos. Podemos lograr esto en Scikit-Learn usando una clase Pipeline. Pipeline ejecuta uno o más modelos dentro de un paso del procedimiento de validación cruzada. Aquí, podemos definir un pipeline con el StandardScaler seguido por nuestro modelo de red neuronal.

In [6]:

import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

seed = 7
numpy.random.seed(seed)


dataframe = pandas.read_csv("sonar.csv", header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]


encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

dummy_y = encoded_Y
# print(dummy_y)


def sonar_model():
    # modelo
    model = Sequential()
    model.add(Dense(60, input_dim=60, init='normal', activation='relu'))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    # compilar
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=sonar_model, epochs=100, batch_size=5)))
pipeline = Pipeline(estimators)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, dummy_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(60, input_dim=60, activation="relu", kernel_initializer="normal")`
C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`


Epoch 1/100
187/187 [==============================] - 2s 10ms/step - loss: 0.6369 - acc: 0.6524
Epoch 2/100
187/187 [==============================] - 0s 1ms/step - loss: 0.5229 - acc: 0.7968
Epoch 3/100
187/187 [==============================] - 0s 997us/step - loss: 0.4304 - acc: 0.8235
Epoch 4/100
187/187 [==============================] - 0s 939us/step - loss: 0.3555 - acc: 0.8556
Epoch 5/100
187/187 [==============================] - 0s 864us/step - loss: 0.2983 - acc: 0.8877
Epoch 6/100
187/187 [==============================] - 0s 901us/step - loss: 0.2560 - acc: 0.8984
Epoch 7/100
187/187 [==============================] - 0s 891us/step - loss: 0.2210 - acc: 0.9465
Epoch 8/100
187/187 [==============================] - 0s 917us/step - loss: 0.1970 - acc: 0.9305
Epoch 9/100
187/187 [==============================] - 0s 901us/step - loss: 0.1721 - acc: 0.9572
Epoch 10/100
187/187 [==============================] - 0s 827us/step - loss: 0.1508 - acc: 0.9679
Epoch 11/100
187/187 [

Epoch 84/100
187/187 [==============================] - 0s 827us/step - loss: 0.0012 - acc: 1.0000
Epoch 85/100
187/187 [==============================] - 0s 816us/step - loss: 0.0012 - acc: 1.0000
Epoch 86/100
187/187 [==============================] - 0s 821us/step - loss: 0.0012 - acc: 1.0000
Epoch 87/100
187/187 [==============================] - 0s 821us/step - loss: 0.0011 - acc: 1.0000
Epoch 88/100
187/187 [==============================] - 0s 805us/step - loss: 0.0011 - acc: 1.0000
Epoch 89/100
187/187 [==============================] - 0s 800us/step - loss: 0.0010 - acc: 1.0000
Epoch 90/100
187/187 [==============================] - 0s 816us/step - loss: 0.0010 - acc: 1.0000
Epoch 91/100
187/187 [==============================] - 0s 821us/step - loss: 9.8029e-04 - acc: 1.0000
Epoch 92/100
187/187 [==============================] - 0s 827us/step - loss: 9.4340e-04 - acc: 1.0000
Epoch 93/100
187/187 [==============================] - 0s 837us/step - loss: 9.2144e-04 - acc: 1.000

C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(60, input_dim=60, activation="relu", kernel_initializer="normal")`
C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`


Epoch 1/100
187/187 [==============================] - 1s 6ms/step - loss: 0.6596 - acc: 0.6524
Epoch 2/100
187/187 [==============================] - 0s 816us/step - loss: 0.5632 - acc: 0.7433
Epoch 3/100
187/187 [==============================] - 0s 843us/step - loss: 0.4626 - acc: 0.8075
Epoch 4/100
187/187 [==============================] - 0s 816us/step - loss: 0.3849 - acc: 0.8610
Epoch 5/100
187/187 [==============================] - 0s 827us/step - loss: 0.3253 - acc: 0.8717
Epoch 6/100
187/187 [==============================] - 0s 821us/step - loss: 0.2799 - acc: 0.8930
Epoch 7/100
187/187 [==============================] - 0s 827us/step - loss: 0.2443 - acc: 0.9198
Epoch 8/100
187/187 [==============================] - 0s 832us/step - loss: 0.2118 - acc: 0.9519
Epoch 9/100
187/187 [==============================] - 0s 837us/step - loss: 0.1897 - acc: 0.9519
Epoch 10/100
187/187 [==============================] - 0s 805us/step - loss: 0.1658 - acc: 0.9733
Epoch 11/100
187/187 

...

Epoch 1/100
187/187 [==============================] - 2s 11ms/step - loss: 0.6492 - acc: 0.6417
Epoch 2/100
187/187 [==============================] - 0s 965us/step - loss: 0.5464 - acc: 0.7701
Epoch 3/100
187/187 [==============================] - 0s 821us/step - loss: 0.4427 - acc: 0.8235
Epoch 4/100
187/187 [==============================] - 0s 827us/step - loss: 0.3659 - acc: 0.8396
Epoch 5/100
187/187 [==============================] - 0s 955us/step - loss: 0.3025 - acc: 0.8930
Epoch 6/100
187/187 [==============================] - 0s 848us/step - loss: 0.2598 - acc: 0.9198
Epoch 7/100
187/187 [==============================] - 0s 965us/step - loss: 0.2247 - acc: 0.9305
Epoch 8/100
187/187 [==============================] - 0s 955us/step - loss: 0.1911 - acc: 0.9572
Epoch 9/100
187/187 [==============================] - 0s 880us/step - loss: 0.1680 - acc: 0.9519
Epoch 10/100
187/187 [==============================] - 0s 933us/step - loss: 0.1430 - acc: 0.9786
Epoch 11/100
187/187

C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(60, input_dim=60, activation="relu", kernel_initializer="normal")`
C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`


Epoch 1/100
188/188 [==============================] - 1s 7ms/step - loss: 0.6608 - acc: 0.6543
Epoch 2/100
188/188 [==============================] - 0s 875us/step - loss: 0.5530 - acc: 0.7872
Epoch 3/100
188/188 [==============================] - 0s 859us/step - loss: 0.4440 - acc: 0.8298
Epoch 4/100
188/188 [==============================] - 0s 881us/step - loss: 0.3686 - acc: 0.8617
Epoch 5/100
188/188 [==============================] - 0s 854us/step - loss: 0.3125 - acc: 0.8830
Epoch 6/100
188/188 [==============================] - 0s 854us/step - loss: 0.2749 - acc: 0.9043
Epoch 7/100
188/188 [==============================] - 0s 886us/step - loss: 0.2392 - acc: 0.9202
Epoch 8/100
188/188 [==============================] - 0s 891us/step - loss: 0.2135 - acc: 0.9309
Epoch 9/100
188/188 [==============================] - 0s 897us/step - loss: 0.1880 - acc: 0.9521
Epoch 10/100
188/188 [==============================] - 0s 854us/step - loss: 0.1671 - acc: 0.9628
Epoch 11/100
188/188 

C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(60, input_dim=60, activation="relu", kernel_initializer="normal")`
C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`


Epoch 1/100
188/188 [==============================] - 1s 7ms/step - loss: 0.6595 - acc: 0.6596
Epoch 2/100
188/188 [==============================] - 0s 881us/step - loss: 0.5662 - acc: 0.8032
Epoch 3/100
188/188 [==============================] - 0s 886us/step - loss: 0.4745 - acc: 0.8245
Epoch 4/100
188/188 [==============================] - 0s 918us/step - loss: 0.3941 - acc: 0.8617
Epoch 5/100
188/188 [==============================] - 0s 886us/step - loss: 0.3311 - acc: 0.8670
Epoch 6/100
188/188 [==============================] - 0s 854us/step - loss: 0.2885 - acc: 0.9043
Epoch 7/100
188/188 [==============================] - 0s 881us/step - loss: 0.2468 - acc: 0.9415
Epoch 8/100
188/188 [==============================] - 0s 875us/step - loss: 0.2167 - acc: 0.9574
Epoch 9/100
188/188 [==============================] - 0s 843us/step - loss: 0.1887 - acc: 0.9628
Epoch 10/100
188/188 [==============================] - 0s 870us/step - loss: 0.1648 - acc: 0.9734
Epoch 11/100
188/188 

# Sintonización de neuronas y capas en el modelo

Hay muchas cosas que sintonizar en una red neuronal, como la inicialización del peso, las funciones de activación, el procedimiento de optimización, etc. Un aspecto que puede tener un efecto de gran tamaño es la estructura de la red en sí misma, llamada topología de la red. Revisemos dos experimentos sobre la estructura de la red: hacerla más pequeña y hacerla más grande. Estos son buenos experimentos para cuando desee sintonizar una red neuronal.

# Evaluar una red más pequeña

Se sospecha que hay mucha redundancia en las variables de entrada para este problema. Los datos describen la misma señal desde diferentes ángulos. Quizás algunos de esos ángulos sean más relevantes que otros. Podemos forzar un tipo de extracción de características por parte de la red al restringir el espacio de representación en la primera capa oculta.

En este experimento tomamos nuestro modelo de referencia con 60 neuronas en la capa oculta y lo reducimos a la mitad a 30. Esto presionará a la red durante el entrenamiento para seleccionar la estructura más importante en los datos de entrada para modelar. También estandarizaremos los datos como en el experimento anterior con la preparación de datos e intentaremos aprovechar el pequeño aumento en el rendimiento.

In [7]:

import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# seed
seed = 7
numpy.random.seed(seed)

## datos 
dataframe = pandas.read_csv("sonar.csv", header=None)
dataset = dataframe.values
# entradas y salidas
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

# codificar
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

dummy_y = encoded_Y
# print(dummy_y)


def modelo_pequeño():
    # modelo
    model = Sequential()
    model.add(Dense(30, input_dim=60, init='normal', activation='relu'))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    # compilar
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=modelo_pequeño, epochs=100, batch_size=5)))
pipeline = Pipeline(estimators)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, dummy_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(30, input_dim=60, activation="relu", kernel_initializer="normal")`
C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`


Epoch 1/100
187/187 [==============================] - 1s 8ms/step - loss: 0.6718 - acc: 0.6524
Epoch 2/100
187/187 [==============================] - 0s 939us/step - loss: 0.5958 - acc: 0.7754
Epoch 3/100
187/187 [==============================] - 0s 912us/step - loss: 0.5052 - acc: 0.8128
Epoch 4/100
187/187 [==============================] - 0s 907us/step - loss: 0.4224 - acc: 0.8342
Epoch 5/100
187/187 [==============================] - 0s 891us/step - loss: 0.3579 - acc: 0.8503
Epoch 6/100
187/187 [==============================] - 0s 928us/step - loss: 0.3140 - acc: 0.8717
Epoch 7/100
187/187 [==============================] - 0s 1ms/step - loss: 0.2791 - acc: 0.9037
Epoch 8/100
187/187 [==============================] - 0s 997us/step - loss: 0.2545 - acc: 0.9037
Epoch 9/100
187/187 [==============================] - 0s 896us/step - loss: 0.2296 - acc: 0.9358
Epoch 10/100
187/187 [==============================] - 0s 891us/step - loss: 0.2094 - acc: 0.9412
Epoch 11/100
187/187 [=

C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(30, input_dim=60, activation="relu", kernel_initializer="normal")`
C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`


Epoch 1/100
187/187 [==============================] - 1s 7ms/step - loss: 0.6731 - acc: 0.6471
Epoch 2/100
187/187 [==============================] - 0s 880us/step - loss: 0.6098 - acc: 0.7701
Epoch 3/100
187/187 [==============================] - 0s 917us/step - loss: 0.5247 - acc: 0.7968
Epoch 4/100
187/187 [==============================] - 0s 901us/step - loss: 0.4451 - acc: 0.8342
Epoch 5/100
187/187 [==============================] - 0s 891us/step - loss: 0.3838 - acc: 0.8449
Epoch 6/100
187/187 [==============================] - 0s 901us/step - loss: 0.3383 - acc: 0.8556
Epoch 7/100
187/187 [==============================] - 0s 917us/step - loss: 0.3026 - acc: 0.8824
Epoch 8/100
187/187 [==============================] - 0s 880us/step - loss: 0.2716 - acc: 0.8984
Epoch 9/100
187/187 [==============================] - 0s 891us/step - loss: 0.2492 - acc: 0.9091
Epoch 10/100
187/187 [==============================] - 0s 848us/step - loss: 0.2269 - acc: 0.9465
Epoch 11/100
187/187 

C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(30, input_dim=60, activation="relu", kernel_initializer="normal")`
C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`


...

C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(30, input_dim=60, activation="relu", kernel_initializer="normal")`
C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`


Epoch 1/100
188/188 [==============================] - 2s 9ms/step - loss: 0.6726 - acc: 0.6543
Epoch 2/100
188/188 [==============================] - 0s 944us/step - loss: 0.6042 - acc: 0.7819
Epoch 3/100
188/188 [==============================] - 0s 934us/step - loss: 0.5171 - acc: 0.8138
Epoch 4/100
188/188 [==============================] - 0s 918us/step - loss: 0.4418 - acc: 0.8457
Epoch 5/100
188/188 [==============================] - 0s 907us/step - loss: 0.3799 - acc: 0.8564
Epoch 6/100
188/188 [==============================] - 0s 944us/step - loss: 0.3360 - acc: 0.8777
Epoch 7/100
188/188 [==============================] - 0s 923us/step - loss: 0.2973 - acc: 0.8936
Epoch 8/100
188/188 [==============================] - 0s 896us/step - loss: 0.2701 - acc: 0.9043
Epoch 9/100
188/188 [==============================] - 0s 934us/step - loss: 0.2423 - acc: 0.9149
Epoch 10/100
188/188 [==============================] - 0s 923us/step - loss: 0.2200 - acc: 0.9309
Epoch 11/100
188/188 

Epoch 84/100
188/188 [==============================] - 0s 886us/step - loss: 0.0022 - acc: 1.0000
Epoch 85/100
188/188 [==============================] - 0s 865us/step - loss: 0.0022 - acc: 1.0000
Epoch 86/100
188/188 [==============================] - 0s 897us/step - loss: 0.0021 - acc: 1.0000
Epoch 87/100
188/188 [==============================] - 0s 881us/step - loss: 0.0020 - acc: 1.0000
Epoch 88/100
188/188 [==============================] - 0s 886us/step - loss: 0.0019 - acc: 1.0000
Epoch 89/100
188/188 [==============================] - 0s 907us/step - loss: 0.0019 - acc: 1.0000
Epoch 90/100
188/188 [==============================] - 0s 912us/step - loss: 0.0018 - acc: 1.0000
Epoch 91/100
188/188 [==============================] - 0s 865us/step - loss: 0.0018 - acc: 1.0000
Epoch 92/100
188/188 [==============================] - 0s 886us/step - loss: 0.0017 - acc: 1.0000
Epoch 93/100
188/188 [==============================] - 0s 875us/step - loss: 0.0017 - acc: 1.0000
Epoch 94/1

C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(30, input_dim=60, activation="relu", kernel_initializer="normal")`
C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`


Epoch 1/100
188/188 [==============================] - 2s 9ms/step - loss: 0.6817 - acc: 0.6011
Epoch 2/100
188/188 [==============================] - 0s 923us/step - loss: 0.6209 - acc: 0.7926
Epoch 3/100
188/188 [==============================] - 0s 955us/step - loss: 0.5415 - acc: 0.8085
Epoch 4/100
188/188 [==============================] - 0s 944us/step - loss: 0.4659 - acc: 0.8404
Epoch 5/100
188/188 [==============================] - 0s 918us/step - loss: 0.4051 - acc: 0.8511
Epoch 6/100
188/188 [==============================] - 0s 891us/step - loss: 0.3626 - acc: 0.8564
Epoch 7/100
188/188 [==============================] - 0s 844us/step - loss: 0.3224 - acc: 0.8830
Epoch 8/100
188/188 [==============================] - 0s 817us/step - loss: 0.2944 - acc: 0.8989
Epoch 9/100
188/188 [==============================] - 0s 902us/step - loss: 0.2668 - acc: 0.9255
Epoch 10/100
188/188 [==============================] - 0s 902us/step - loss: 0.2440 - acc: 0.9362
Epoch 11/100
188/188 


# Evaluar una red más grande

Una topología de red neuronal con más capas ofrece más oportunidades para que la red extraiga características claves y las combine de formas no lineales útiles. Podemos evaluar si agregar más capas a la red mejora el rendimiento haciendo otro pequeño ajuste a la función utilizada para crear nuestro modelo. Aquí, agregamos una nueva capa a la red que introduce otra capa oculta con 30 neuronas después de la primera capa oculta. Nuestra red ahora tiene la topología:

60 entradas -> [60 -> 30] -> 1 salida

La idea aquí es que a la red se le da la oportunidad de modelar todas las variables de entrada antes de ser estancada y obligada a reducir a la mitad la capacidad de representación, como lo hicimos en el experimento anterior con la red más pequeña. En lugar de exprimir la representación de las entradas en sí, tenemos una capa oculta adicional para ayudar en el proceso.

In [9]:
# 
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# 
seed = 7
numpy.random.seed(seed)

#
dataframe = pandas.read_csv("sonar.csv", header=None)
dataset = dataframe.values
# 
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

# 
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

dummy_y = encoded_Y
# print(dummy_y)

# 
def modelo_grande():
    # 
    model = Sequential()
    model.add(Dense(60, input_dim=60, init='normal', activation='relu'))
    model.add(Dense(30, init='normal', activation='relu'))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    # 
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=modelo_grande, epochs=100, batch_size=5)))
pipeline = Pipeline(estimators)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, dummy_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(60, input_dim=60, activation="relu", kernel_initializer="normal")`
C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(30, activation="relu", kernel_initializer="normal")`
C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:38: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`


Epoch 1/100
187/187 [==============================] - 2s 12ms/step - loss: 0.6864 - acc: 0.6578
Epoch 2/100
187/187 [==============================] - 0s 1ms/step - loss: 0.6092 - acc: 0.8235
Epoch 3/100
187/187 [==============================] - 0s 1ms/step - loss: 0.4300 - acc: 0.8182
Epoch 4/100
187/187 [==============================] - 0s 1ms/step - loss: 0.2811 - acc: 0.8930
Epoch 5/100
187/187 [==============================] - 0s 987us/step - loss: 0.1995 - acc: 0.9305
Epoch 6/100
187/187 [==============================] - 0s 1ms/step - loss: 0.1457 - acc: 0.9572
Epoch 7/100
187/187 [==============================] - 0s 1ms/step - loss: 0.1166 - acc: 0.9733
Epoch 8/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0757 - acc: 0.9893
Epoch 9/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0552 - acc: 0.9947
Epoch 10/100
187/187 [==============================] - 0s 987us/step - loss: 0.0351 - acc: 1.0000
Epoch 11/100
187/187 [============

C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(60, input_dim=60, activation="relu", kernel_initializer="normal")`
C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(30, activation="relu", kernel_initializer="normal")`
C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:38: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`


Epoch 1/100
187/187 [==============================] - 2s 12ms/step - loss: 0.6809 - acc: 0.6524
Epoch 2/100
187/187 [==============================] - 0s 1ms/step - loss: 0.5964 - acc: 0.7487
Epoch 3/100
187/187 [==============================] - 0s 1ms/step - loss: 0.4318 - acc: 0.8182
Epoch 4/100
187/187 [==============================] - 0s 1ms/step - loss: 0.3191 - acc: 0.8717
Epoch 5/100
187/187 [==============================] - 0s 1ms/step - loss: 0.2432 - acc: 0.9144
Epoch 6/100
187/187 [==============================] - 0s 1ms/step - loss: 0.1853 - acc: 0.9358
Epoch 7/100
187/187 [==============================] - 0s 1ms/step - loss: 0.1403 - acc: 0.9733
Epoch 8/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0970 - acc: 0.9840
Epoch 9/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0713 - acc: 0.9893
Epoch 10/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0484 - acc: 0.9893
Epoch 11/100
187/187 [================

C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(60, input_dim=60, activation="relu", kernel_initializer="normal")`
C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(30, activation="relu", kernel_initializer="normal")`
C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:38: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`


...

C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(60, input_dim=60, activation="relu", kernel_initializer="normal")`
C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(30, activation="relu", kernel_initializer="normal")`
C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:38: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`


Epoch 1/100
188/188 [==============================] - 2s 11ms/step - loss: 0.6726 - acc: 0.7021
Epoch 2/100
188/188 [==============================] - 0s 1ms/step - loss: 0.5684 - acc: 0.7872
Epoch 3/100
188/188 [==============================] - 0s 1ms/step - loss: 0.3928 - acc: 0.8404
Epoch 4/100
188/188 [==============================] - 0s 1ms/step - loss: 0.2825 - acc: 0.8883
Epoch 5/100
188/188 [==============================] - 0s 1ms/step - loss: 0.2157 - acc: 0.9255
Epoch 6/100
188/188 [==============================] - 0s 1ms/step - loss: 0.1649 - acc: 0.9521
Epoch 7/100
188/188 [==============================] - 0s 1ms/step - loss: 0.1165 - acc: 0.9734
Epoch 8/100
188/188 [==============================] - 0s 1ms/step - loss: 0.0807 - acc: 0.9947
Epoch 9/100
188/188 [==============================] - 0s 1ms/step - loss: 0.0545 - acc: 0.9894
Epoch 10/100
188/188 [==============================] - 0s 1ms/step - loss: 0.0358 - acc: 1.0000
Epoch 11/100
188/188 [================

188/188 [==============================] - 0s 971us/step - loss: 7.0818e-05 - acc: 1.0000
Epoch 84/100
188/188 [==============================] - 0s 1ms/step - loss: 6.8561e-05 - acc: 1.0000
Epoch 85/100
188/188 [==============================] - 0s 992us/step - loss: 6.6436e-05 - acc: 1.0000
Epoch 86/100
188/188 [==============================] - 0s 1ms/step - loss: 6.4259e-05 - acc: 1.0000
Epoch 87/100
188/188 [==============================] - 0s 1ms/step - loss: 6.2104e-05 - acc: 1.0000
Epoch 88/100
188/188 [==============================] - 0s 981us/step - loss: 6.0416e-05 - acc: 1.0000
Epoch 89/100
188/188 [==============================] - 0s 1ms/step - loss: 5.8189e-05 - acc: 1.0000
Epoch 90/100
188/188 [==============================] - 0s 960us/step - loss: 5.6458e-05 - acc: 1.0000
Epoch 91/100
188/188 [==============================] - 0s 966us/step - loss: 5.4805e-05 - acc: 1.0000
Epoch 92/100
188/188 [==============================] - 0s 992us/step - loss: 5.3143e-05 - acc

C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(60, input_dim=60, activation="relu", kernel_initializer="normal")`
C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(30, activation="relu", kernel_initializer="normal")`
C:\ProgramData\Anaconda3\envs\tensorgpu\lib\site-packages\ipykernel_launcher.py:38: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`


Epoch 1/100
188/188 [==============================] - 2s 11ms/step - loss: 0.6812 - acc: 0.7021
Epoch 2/100
188/188 [==============================] - 0s 1ms/step - loss: 0.6108 - acc: 0.7660
Epoch 3/100
188/188 [==============================] - 0s 1ms/step - loss: 0.4680 - acc: 0.8138
Epoch 4/100
188/188 [==============================] - 0s 1ms/step - loss: 0.3531 - acc: 0.8617
Epoch 5/100
188/188 [==============================] - 0s 1ms/step - loss: 0.2654 - acc: 0.8989
Epoch 6/100
188/188 [==============================] - 0s 1ms/step - loss: 0.1959 - acc: 0.9521
Epoch 7/100
188/188 [==============================] - 0s 1ms/step - loss: 0.1441 - acc: 0.9521
Epoch 8/100
188/188 [==============================] - 0s 1ms/step - loss: 0.1060 - acc: 0.9681
Epoch 9/100
188/188 [==============================] - 0s 1ms/step - loss: 0.0680 - acc: 0.9947
Epoch 10/100
188/188 [==============================] - 0s 1ms/step - loss: 0.0486 - acc: 0.9894
Epoch 11/100
188/188 [================